<a href="https://colab.research.google.com/github/ohaanika/classify-doodles/blob/master/model_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf

TRAIN_DIR = '../datasets/train_images.npy'
TRAIN_LABEL_DIR = '../datasets/train_labels.csv'
TEST_DIR = '../datasets/test_images.npy'
TRAIN_DIR_PRE = '../datasets/train_images_cropped.npy'
TEST_DIR_PRE = '../datasets/test_images_cropped.npy'
IMG_SIZE = 100
LR = 1e-4

MODEL_NAME  = 'hand-drawn-{}-{}.model'.format(LR, '2conv-basic')

In [3]:
cate = ['sink','pear','moustache','nose','skateboard','penguin'
              ,'peanut','skull','panda','paintbrush','nail','apple',
              'rifle','mug','sailboat','pineapple','spoon','rabbit',
              'shovel','rollerskates','screwdriver','scorpion','rhinoceros'
              ,'pool','octagon','pillow','parrot','squiggle','mouth',
               'empty','pencil']
categories = {0:'apple',1:'empty',2:'moustache', 3:'mouth', 4:'mug', 5:'nail', 6:'nose'\
              ,7:'octagon', 8:'paintbrush', 9:'panda', 10:'parrot',11:'peanut',\
              12:'pear', 13:'pencil', 14:'penguin',15:'pillow',16:'pineapple',17:'pool'\
              ,18:'rabbit', 19:'rhinoceros',20:'rifle',21:'rollerskates',22:'sailboat',\
              23:'scorpion',24:'screwdriver', 25:'shovel', 26:'sink', 27:'skateboard',\
              28:'skull',29:'spoon',30:'squiggle'}
from sklearn import preprocessing
#This will give a 1-of-k coding scheme with alphbat sequence
lb = preprocessing.LabelBinarizer()
lb.fit(cate)

[['sink', 0], ['pear', 1], ['moustache', 2], ['nose', 3], ['skateboard', 4], ['penguin', 5], ['peanut', 6], ['skull', 7], ['panda', 8], ['paintbrush', 9], ['nail', 10], ['apple', 11], ['rifle', 12], ['mug', 13], ['sailboat', 14], ['pineapple', 15], ['spoon', 16], ['rabbit', 17], ['shovel', 18], ['rollerskates', 19], ['screwdriver', 20], ['scorpion', 21], ['rhinoceros', 22], ['pool', 23], ['octagon', 24], ['pillow', 25], ['parrot', 26], ['squiggle', 27], ['mouth', 28], ['empty', 29], ['pencil', 30]]


array([0])

In [88]:
#Notice here preprocessed image as input
train_row = np.load(TRAIN_DIR_PRE, encoding = 'latin1')
train_label = np.array(pd.read_csv(TRAIN_LABEL_DIR, delimiter=","))
final_test = np.load(TEST_DIR_PRE,encoding='latin1')
#train = train_data[:7000]
#valid = train_data[7000:9000]
#test = train_data[9000:]
train = train_data[:9000]
valid = train_data[9000:9950]
test = train_data[9950:]

In [89]:
train_labels = train_label[:,1]
train_x = train_row[:,1]
train_data = []
for i in range(len(train_labels)):
    encoded = lb.transform([train_labels[i]])
    train_data.append([np.array(train_x[i]), encoded[0]])

In [20]:
tf.reset_default_graph()
convent = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1],name = 'input')

convent = conv_2d(convent, nb_filter = 40, filter_size = [5,5], activation = 'relu')
convent = max_pool_2d(convent, 2)

convent = conv_2d(convent, nb_filter = 64, filter_size = [2,2], activation = 'relu')
convent = max_pool_2d(convent, 2)

convent = conv_2d(convent, nb_filter = 20, filter_size = [2,2], activation = 'relu')
convent = max_pool_2d(convent, 2)

#convent = fully_connected(convent, 1024, activation = 'relu')
convent = dropout(convent, 0.8)

convent = fully_connected(convent, 31, activation ='softmax')
convent = regression(convent, optimizer = 'Adam', learning_rate = LR, loss='categorical_crossentropy', name = 'target')

model = tflearn.DNN(convent, tensorboard_dir = 'log')

In [37]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [90]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]
Y = np.reshape(Y,(-1,31))


valid_x = np.array([i[0] for i in valid]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
valid_y = np.array([i[1] for i in valid]).reshape(-1,31)

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = np.array([i[1] for i in test]).reshape(-1,31)

# the shape of preprocessed test set is (10000,2) idx 0: img, idx 1:id
final_test = final_test[:,1]
final_test_x = np.array([i for i in final_test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [21]:
model.fit(X, Y, n_epoch = 15, validation_set = (valid_x,valid_y),
          snapshot_step=200, show_metric=True, run_id=MODEL_NAME)

Training Step: 1649  | total loss: 0.88002 | time: 31.743s
| Adam | epoch: 015 | loss: 0.88002 - acc: 0.7278 -- iter: 6976/7000
Training Step: 1650  | total loss: 0.89207 | time: 36.411s
| Adam | epoch: 015 | loss: 0.89207 - acc: 0.7206 | val_loss: 8.67389 - val_acc: 0.0665 -- iter: 7000/7000
--


In [27]:
model.save(MODEL_NAME)

INFO:tensorflow:/home/ubuntu/Documents/comp551/classify-doodles/hand-drawn-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [71]:
acc = 0
for num, data in enumerate(test):
    img_num = data[1]
    img_data = data[0]
  
    orig = img_data.reshape(IMG_SIZE, IMG_SIZE)
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
  
    model_out = cnn_model.predict([data])[0]  
    prediction = model_out.argmax(axis=-1)
  
    true = img_num.argmax(axis=-1)
    if(prediction == true): acc+=1
result = acc/(len(test))
print(result)

0.584


In [37]:
from sklearn.metrics import accuracy_score

In [72]:
pred_test = cnn_model.predict(final_test_x[:3000]).argmax(axis=-1)

In [73]:
pred_test_p2 = cnn_model.predict(final_test_x[3000:6000]).argmax(axis=-1)
pred_test_p3 = cnn_model.predict(final_test_x[6000:]).argmax(axis=-1)

In [74]:
pred_test = np.hstack([pred_test,pred_test_p2,pred_test_p3])

In [76]:
import csv
with open("../output/submit_03.csv","w") as f:
    wt = csv.writer(f,delimiter=',')
    wt.writerow(['Id','Category'])
    for idx,i in enumerate(pred_test):
        wt.writerow([str(idx),str(categories.get(i))])

In [92]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasClassifier
import pprint
def cnn_tuning(rate_num, l_rate):
    tf.reset_default_graph()
    convent = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1],name = 'input')
    convent = conv_2d(incoming = convent, nb_filter = 32, filter_size = [5,5], padding ='valid', activation ='relu')
    convent = max_pool_2d(incoming = convent, kernel_size = [2,2], strides =[2,2])
    convent = conv_2d(incoming = convent, nb_filter = 64, filter_size = [5,5], padding ='valid', activation ='relu')
    convent = max_pool_2d(incoming = convent, kernel_size = [2,2], strides =[2,2])
    convent = conv_2d(incoming = convent, nb_filter = 32, filter_size = [5,5], padding ='valid', activation ='relu')
    convent = max_pool_2d(incoming = convent, kernel_size = [2,2], strides =[2,2])
    convent = conv_2d(incoming = convent, nb_filter = 64, filter_size = [5,5], padding ='valid', activation ='relu')
    convent = max_pool_2d(incoming = convent, kernel_size = [2,2], strides =[2,2])
    #convent = conv_2d(incoming = convent, nb_filter = 64, filter_size = [3,3], padding ='valid', activation ='relu')
    #convent = avg_pool_2d(incoming = convent, kernel_size = [2,2], strides =[2,2])
    #convent = conv_2d(incoming = convent, nb_filter = 64, filter_size = [3,3], padding ='valid', activation ='relu')
    #convent = avg_pool_2d(incoming = convent, kernel_size = [2,2], strides =[2,2])
    
    dropout = tf.layers.dropout(inputs = convent, rate =rate_num)
    convent = fully_connected(convent, 31, activation = 'softmax')
    convent = regression(convent, optimizer = 'adam', learning_rate = l_rate, loss = 'categorical_crossentropy', name = 'target')
    model = tflearn.DNN(convent, tensorboard_dir = 'log')
    return model 

In [93]:
cnn_model = cnn_tuning(0.3,1e-4)
cnn_model.fit(X, Y, n_epoch = 15, validation_set = (valid_x,valid_y),
              snapshot_step=200, show_metric=True, run_id=MODEL_NAME)

Training Step: 2114  | total loss: 0.63017 | time: 20.852s
| Adam | epoch: 015 | loss: 0.63017 - acc: 0.8433 -- iter: 8960/9000
Training Step: 2115  | total loss: 0.63925 | time: 22.037s
| Adam | epoch: 015 | loss: 0.63925 - acc: 0.8371 | val_loss: 2.24005 - val_acc: 0.5768 -- iter: 9000/9000
--


In [ ]:
nb1 = [32, 64]

In [57]:
dropout_rate = [0.1, 0.3, 0.5, 0.7, 0.9]
with open('result_dropout.csv','w') as f:
    f.write("dropout,accuracy")
    f.write('\n')
    for element in dropout_rate:
        cnn_model = cnn_tuning(element,1e-4)
        cnn_model.fit(X, Y, n_epoch = 8, validation_set = (valid_x,valid_y),
                  snapshot_step=200, show_metric=True, run_id=MODEL_NAME)
        result = cnn_model.evaluate(test_x, test_y)
        f.write(str(element))
        f.write(',')
        f.write(str(result))
        f.write('\n')

Training Step: 879  | total loss: 0.23962 | time: 10.545s
| Adam | epoch: 008 | loss: 0.23962 - acc: 0.9592 -- iter: 6976/7000
Training Step: 880  | total loss: 0.26303 | time: 11.668s
| Adam | epoch: 008 | loss: 0.26303 - acc: 0.9601 | val_loss: 3.52627 - val_acc: 0.5340 -- iter: 7000/7000
--


In [ ]:
epoch = [8, 12, 15, 20, 25]
with open('result_epoch.csv','w') as f:
    f.write("epoch,accuracy")
    f.write('\n')
    for element in epoch:
        cnn_model = cnn_tuning(0.3, 1e-4)
        cnn_model.fit(X, Y, n_epoch = element, validation_set = (valid_x,valid_y),
                  snapshot_step=200, show_metric=True, run_id=MODEL_NAME)
        result = cnn_model.evaluate(test_x, test_y)
        f.write(str(element))
        f.write(',')
        f.write(str(result))
        f.write('\n')

Training Step: 651  | total loss: 0.38218 | time: 9.786s
| Adam | epoch: 006 | loss: 0.38218 - acc: 0.9155 -- iter: 6464/7000


In [11]:
snapshot_step = [50, 100, 150, 200, 300]
with open('result_snapshot_step.csv','w') as f:
    f.write("snapshot_step,accuracy")
    f.write('\n')
    for element in snapshot_step:
        cnn_model = cnn_tuning(0.3, 1e-4)
        cnn_model.fit(X, Y, n_epoch = 25, validation_set = (valid_x,valid_y),
                  snapshot_step=element, show_metric=True, run_id=MODEL_NAME)
        result = cnn_model.evaluate(test_x, test_y)
        f.write(str(element))
        f.write(',')
        f.write(str(result))
        f.write('\n')

Training Step: 2749  | total loss: 0.03061 | time: 10.395s
| Adam | epoch: 025 | loss: 0.03061 - acc: 0.9962 -- iter: 6976/7000
Training Step: 2750  | total loss: 0.03175 | time: 11.512s
| Adam | epoch: 025 | loss: 0.03175 - acc: 0.9950 | val_loss: 5.20567 - val_acc: 0.5425 -- iter: 7000/7000
--


In [14]:
learning_rate = [1e-3, 1e-4, 1e-5]
with open('result_learning_rate.csv','w') as f:
    f.write("learning_rate,accuracy")
    f.write('\n')
    for element in learning_rate:
        cnn_model = cnn_tuning(0.3, element)
        cnn_model.fit(X, Y, n_epoch = 25, validation_set = (valid_x,valid_y),
                  snapshot_step=200, show_metric=True, run_id=MODEL_NAME)
        result = cnn_model.evaluate(test_x, test_y)
        f.write(str(element))
        f.write(',')
        f.write(str(result))
        f.write('\n')

Training Step: 2749  | total loss: 2.53048 | time: 9.509s
| Adam | epoch: 025 | loss: 2.53048 - acc: 0.6897 -- iter: 6976/7000
Training Step: 2750  | total loss: 2.54260 | time: 10.634s
| Adam | epoch: 025 | loss: 2.54260 - acc: 0.6958 | val_loss: 5.34064 - val_acc: 0.4130 -- iter: 7000/7000
--


In [34]:
batch_size = [20, 50, 100, 200]
epochs = [5, 10, 15, 20]
initilizer = ['lecun_uniform', 'normal', 'he_normal', 'he_uniform']
learning_rate = [0.01, 0.1, 0.3]
dropout_rate = [0.1, 0.3, 0.5, 0.7, 0.9]
num_unit = [5,10]
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid','linear']
optimizer = ['SGD', 'Adam', 'Adamax', 'Adagrad']

# parameters = dict(batch_size = batch_size, epochs = epochs, 
#                  dropout_rate = dropout_rate, num_unit = num_unit,
#                  initilizer = initilizer, learning_rate = learning_rate,
#                  activation = activation, optimizer = optimizer)
parameters = dict(dropout_rate= dropout_rate)

In [35]:
from sklearn.model_selection import GridSearchCV
#new_model = KerasClassifier(build_fn = build_model, verbose =0)
models = GridSearchCV(estimator = cnn_model, param_grid = parameters, scoring = 'accuracy')